In [1]:

import sqlite3

# Connessione al database
conn = sqlite3.connect('linkedin.db')
cur = conn.cursor()

# Creazione delle tabelle
tables = {
    "groups": {
        "columns": {
            "group_id": "INTEGER PRIMARY KEY",
            "group_name": "TEXT NOT NULL",
            "group_description": "TEXT",
        },
    },
    "connection": {
        "columns": {
            "user_id": "INTEGER NOT NULL",
            "user_id_2": "INTEGER NOT NULL",
            "type": "TEXT NOT NULL CHECK (type IN ('follow', 'connect'))",
        },
        "foreign_keys": [
            ("user_id", "user(user_id)"),
            ("user_id_2", "user(user_id)"),
        ],
        "checks": [
            "user_id < user_id_2"
        ]
    },
    "membership": {
        "columns": {
            "group_id": "INTEGER NOT NULL",
            "user_id": "INTEGER NOT NULL",
            "membership_date": "DATE",
        },
        "foreign_keys": [
            ("group_id", "groups(group_id)"),
            ("user_id", "user(user_id)"),
        ],
    },
    "user": {
        "columns": {
            "user_id": "INTEGER PRIMARY KEY",
            "user_type": "TEXT NOT NULL CHECK (user_type IN ('PEOPLE', 'COMPANY_INSTITUTE'))",
            "email": "TEXT NOT NULL UNIQUE",
            "pwd": "TEXT NOT NULL",
        },
    },
    "people": {
        "columns": {
            "user_id": "INTEGER PRIMARY KEY",
            "first_name": "TEXT NOT NULL",
            "last_name": "TEXT NOT NULL",
            "age": "INTEGER",
            "gender": "TEXT",
        },
        "foreign_keys": [
            ("user_id", "user(user_id)"),
        ]
    },
    "company_inst": {
        "columns": {
            "user_id_company_inst": "INTEGER PRIMARY KEY",
            "name": "TEXT NOT NULL",
            "address": "TEXT NOT NULL",
            "industry": "TEXT",
        },
        "foreign_keys": [
            ("user_id_company_inst", "user(user_id)"),
        ]
    },
    "education": {
        "columns": {
            "user_id_people": "INTEGER",
            "user_id_company_inst": "INTEGER",
            "degree": "TEXT",
        },
        "foreign_keys": [
            ("user_id_people", "people(user_id)"),
            ("user_id_company_inst", "company_inst(user_id_company_inst)"),
        ]
    },
    "work_experience": {
        "columns": {
            "user_id_people": "INTEGER",
            "user_id_company_inst": "INTEGER",
            "position": "TEXT",
            "start_date": "DATE",
            "end_date": "DATE",
        },
        "foreign_keys": [
            ("user_id_people", "people(user_id)"),
            ("user_id_company_inst", "company_inst(user_id_company_inst)"),
        ]
    },
    "apply": {
        "columns": {
            "user_id_people": "INTEGER",
            "user_id_company_inst": "INTEGER",
            "status": "TEXT",
            "position": "TEXT",
        },
        "foreign_keys": [
            ("user_id_people", "people(user_id)"),
            ("user_id_company_inst", "company_inst(user_id_company_inst)"),
        ]
    },
    "endorsement": {
        "columns": {
            "user_id_people_end_1": "INTEGER",
            "user_id_people_end_2": "INTEGER",
            "skill_id": "INTEGER",
        },
        "foreign_keys": [
            ("user_id_people_end_1", "people(user_id)"),
            ("user_id_people_end_2", "people(user_id)"),
            ("skill_id", "skill(skill_id)"),
        ],
        "checks": [
            "user_id_people_end_1 < user_id_people_end_2"
        ]
    },
    "skill": {
        "columns": {
            "skill_id": "TEXT PRIMARY KEY",
            "skill_description": "TEXT",
            "proficiency_level": "TEXT",
        },
    },
    "mention": {
        "columns": {
            "user_id": "INTEGER",
            "post_id": "INTEGER",
        },
        "foreign_keys": [
            ("user_id", "user(user_id)"),
            ("post_id", "post(post_id)"),
        ]
    },
    "main_page": {
        "columns": {
            "user_id": "INTEGER",
            "post_id": "INTEGER",
        },
        "foreign_keys": [
            ("user_id", "user(user_id)"),
            ("post_id", "post(post_id)"),
        ]
    },
    "author": {
        "columns": {
            "user_id": "INTEGER",
            "post_id": "INTEGER",
        },
        "foreign_keys": [
            ("user_id", "user(user_id)"),
            ("post_id", "post(post_id)"),
        ]
    },
    "post": {
        "columns": {
            "post_id": "INTEGER PRIMARY KEY",
            "post_date": "DATE NOT NULL",
            "viewed_by": "INTEGER",
        },
    },
    "interaction": {
        "columns": {
            "post_id": "INTEGER",
            "post_id_2": "INTEGER",
            "content": "TEXT",
        },
        "foreign_keys": [
            ("post_id", "post(post_id)"),
            ("post_id_2", "post(post_id)"),
        ],
        "checks": [
            "post_id < post_id_2"
        ]
    },
    "in_text": {
        "columns": {
            "post_id": "INTEGER",
            "hashtag_id": "INTEGER",
        },
        "foreign_keys": [
            ("post_id", "post(post_id)"),
            ("hashtag_id", "hashtag(hashtag_id)"),
        ]
    },
    "hashtag": {
        "columns": {
            "hashtag_id": "INTEGER PRIMARY KEY",
            "description": "TEXT",
        },
    },
    "private_message": {
        "columns": {
            "post_id_pm": "INTEGER PRIMARY KEY",
            "content_txt": "TEXT",
            "content_image": "BLOB",
        },
    },
    "recipient": {
        "columns": {
            "post_id_pm": "INTEGER",
            "user_id": "INTEGER",
        },
        "foreign_keys": [
            ("post_id_pm", "private_message(post_id_pm)"),
            ("user_id", "user(user_id)"),
        ]
    },
    "action_in_post": {
        "columns": {
            "post_id_aip": "INTEGER PRIMARY KEY",
            "reaction": "TEXT",
        },
    },
    "public_post": {
        "columns": {
            "post_id_pp": "INTEGER PRIMARY KEY",
            "content_txt": "TEXT",
            "content_image": "BLOB",
        },
    },
    "event": {
        "columns": {
            "post_id_ev": "INTEGER PRIMARY KEY",
            "location": "TEXT",
            "description": "TEXT",
            "date": "DATE",
        },
    },
    "job_post": {
        "columns": {
            "post_id_jp": "INTEGER PRIMARY KEY",
            "user_id_company": "INTEGER",
            "job_title": "TEXT",
            "job_description": "TEXT",
        },
        "foreign_keys": [
            ("user_id_company", "company_inst(user_id_company_inst)"),
        ]
    },
}


# Crea le tabelle
for table, props in tables.items():
    cols = props["columns"]
    col_defs = [f"{name} {dtype}" for name, dtype in cols.items()]

    # Aggiungi chiavi esterne se presenti
    if "foreign_keys" in props:
        fks = [
            f"FOREIGN KEY ({col}) REFERENCES {ref} ON DELETE CASCADE ON UPDATE CASCADE"
            for col, ref in props["foreign_keys"]
        ]
        col_defs.extend(fks)

    if "checks" in props:
        checks = [f"CHECK ({cond})" for cond in props["checks"]]
        col_defs.extend(checks)

    # Comando SQL per creare la tabella
    sql = f"CREATE TABLE IF NOT EXISTS {table} (\n  " + ",\n  ".join(col_defs) + "\n);"
    cur.execute(sql)

# Salvataggio modifiche
conn.commit()

# Creazione di indici su colonne comunemente usate per join o ricerche
index_statements = [
    "CREATE INDEX IF NOT EXISTS idx_connection_user1 ON connection(user_id);",
    "CREATE INDEX IF NOT EXISTS idx_connection_user2 ON connection(user_id_2);",
    "CREATE INDEX IF NOT EXISTS idx_membership_user ON membership(user_id);",
    "CREATE INDEX IF NOT EXISTS idx_membership_group ON membership(group_id);",
    "CREATE INDEX IF NOT EXISTS idx_post_viewed_by ON post(viewed_by);",
    "CREATE INDEX IF NOT EXISTS idx_apply_user ON apply(user_id_people);",
    "CREATE INDEX IF NOT EXISTS idx_apply_company ON apply(user_id_company_inst);"
]

for stmt in index_statements:
    cur.execute(stmt)

# Chiusura connessione
conn.close()
